# Sistemas de Recomendação

## Simple Recommender
Para começarmos a entender os sistemas de recomendação, vamos usar como base de comparação a lista de 250 filmes do IMDB, calculados segundo uma métrica específica. Todos os filmes na lista são não-documentário, lançamento de cinema, possui pelo menos 45 minutos e tem mais de 250000 avaliações. 

Você pode consultar essa lista [aqui](https://www.imdb.com/chart/top/)

Como vimos, isso é um tipo de recomendação.  Vamos usar o data set The Movies Dataset [link](https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset) para criar uma lista parecida a fim de oferencer uma recomendação inicial. 

Link para download dos dados: [link](https://drive.google.com/drive/folders/12y6Wa9D4X1pQhCOnGE2DOGvmOvFijv8u?usp=sharing)

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('dados/movies_metadata.csv')
df.head()

/var/folders/8r/76rtgv650dq8578qqnhrpvym0000gn/T/ipykernel_12413/3055881411.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('dados/movies_metadata.csv')


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


Para construir esse simples recommender, vamos seguir os seguintes passos:

1. Escolher uma métrica para avaliar os filmes
2. Decidir os pré-requisitos para o filme fazer parte da lista
3. Calcular o score para cada filme de acordo com a métrica e os pré-requisitos
4. Retornar a lista de filmes em ordem decrescente de acordo com o score

**Escolha da Métrica**

Para a métrica, vamos usar o Weighted Rating, definido da seguinte maneira:

$ WR = (\frac{v}{v+m} \times R) + (\frac{m}{v+m} \times C) $

Em que:

* $v$ é o número de votos que o filme gerou
* $m$ é o número mínumo de votos requerido para o filme fazer parte da lista
* $R$ é o rating médio do filme
* $C$ é o rating médio de todos os filmes no dataset

Já temos os valores de $v$ e $R$ paara todos os filmes através das features *vote_count* e *vote_average*. Vamos calcular o valor de $m$. $m$ pode ser qualquer valor, mas para esse caso, vamos o número de votos acumulado pelo 80th percentil, que pode ser calculado da seguinte forma:

In [2]:
m = df['vote_count'].quantile(0.80)
m #apenas 20% dos filmes ganharam mais que 50 votos

50.0

Vamos filtrar agora filmes que tiveram mais que 45 minutos e menos que 300 minutos de duração. Também vamos considerar apenas aqueles que tiveram mais que $m$ votos

In [3]:
q_movies = df[(df['runtime'] >= 45) & (df['runtime'] <= 300)]
q_movies = q_movies[q_movies['vote_count'] >= m]
q_movies.shape

(8963, 24)

Para calcular o valor de $C$, vamos obter a média da coluna *vote_average*

In [4]:
C = df['vote_average'].mean()
C

5.618207215134185

De posso de todas as informações, podemos escrever uma função para calcular o weighted rating:

In [5]:
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [6]:
#cria uma nova coluna e aplica a funcao
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)

In [7]:
#ordena o dataset em ordem descendente de acordo com o score
q_movies = q_movies.sort_values('score', ascending=False)

#Imprime os top 25 filmes
q_movies[['title', 'vote_count', 'vote_average', 'score', 'runtime']].head(25)

,title,vote_count,vote_average,score,runtime
10309,Dilwale Dulhania Le Jayenge,661.0,9.1,8.855148,190.0
314,The Shawshank Redemption,8358.0,8.5,8.482863,142.0
834,The Godfather,6024.0,8.5,8.476278,175.0
40251,Your Name.,1030.0,8.5,8.366584,106.0
12481,The Dark Knight,12269.0,8.3,8.289115,152.0
2843,Fight Club,9678.0,8.3,8.286216,139.0
292,Pulp Fiction,8670.0,8.3,8.284623,154.0
522,Schindler's List,4436.0,8.3,8.270109,195.0
23673,Whiplash,4376.0,8.3,8.269704,105.0
5481,Spirited Away,3968.0,8.3,8.266628,125.0


## Knowledge-based recommender
Vamos avançar um pouco e criar um sistema um pouco mais complexo para fazer recomendações. Vamos realizar essas tarefas:

* Perguntar ao usuário seus gêneros preferidos
* Perguntar ao usuário a duração do filme
* Perguntar ao usuário a linha do tempo de filmes recomendados
* Usando essas informações coletadas, recomende filmes ao usuário que possuam um alto score e que satisfaça as condições anteriores

In [8]:
#vamos usar o mesmo dataset e manter apenas as features de interesse
df = df[['title','genres', 'release_date', 'runtime', 'vote_average', 'vote_count']]

df.head()

,title,genres,release_date,runtime,vote_average,vote_count
0,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",1995-10-30,81.0,7.7,5415.0
1,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",1995-12-15,104.0,6.9,2413.0
2,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",1995-12-22,101.0,6.5,92.0
3,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",1995-12-22,127.0,6.1,34.0
4,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]",1995-02-10,106.0,5.7,173.0


Vamos extrair o ano de lançamento a partir da coluna *release_date*

In [9]:
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')
df['year'] = df['release_date'].apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

In [10]:
df['year'].dtype

dtype('O')

Percebemos que o tipo de dados é objeto, pois existem valores NaT nos dados. Vamos usar uma função para transformar NaT em 0 e todos os demais valores em int

In [11]:
def convert_int(x):
    try:
        return int(x)
    except:
        return 0
    
df['year'] = df['year'].apply(convert_int)

Por fim, podemos dropar a coluna *release_date*

In [12]:
df = df.drop('release_date', axis=1)
df.head()

,title,genres,runtime,vote_average,vote_count,year
0,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",81.0,7.7,5415.0,1995
1,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",104.0,6.9,2413.0,1995
2,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",101.0,6.5,92.0,1995
3,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",127.0,6.1,34.0,1995
4,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]",106.0,5.7,173.0,1995


Agora, precisamos lidar com a feature *genres*, que a princípio, se parece com um json. Vamos ver um exemplo:

In [13]:
df.iloc[0]['genres']

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

Para que essa feature seja usável, é importante convertermos ela para um dicionário nativo do Python. A função *literal_eval* faz exatamente isso. Veja um exemplo:

In [14]:
from ast import literal_eval

a = "[1,2,3]"
print(type(a))

b = literal_eval(a)
print(type(b))


<class 'str'>
<class 'list'>


Agora, podemos aplicar nos nossos dados

In [15]:
#converte todos os NaN em strings de listas vazias
df['genres'] = df['genres'].fillna('[]')

#Applica literal_eval para converter para list
df['genres'] = df['genres'].apply(literal_eval)

#Converte lista de dicionário em lista de string
df['genres'] = df['genres'].apply(lambda x: [i['name'].lower() for i in x] if isinstance(x, list) else [])


In [16]:
df.head()

,title,genres,runtime,vote_average,vote_count,year
0,Toy Story,"[animation, comedy, family]",81.0,7.7,5415.0,1995
1,Jumanji,"[adventure, fantasy, family]",104.0,6.9,2413.0,1995
2,Grumpier Old Men,"[romance, comedy]",101.0,6.5,92.0,1995
3,Waiting to Exhale,"[comedy, drama, romance]",127.0,6.1,34.0,1995
4,Father of the Bride Part II,[comedy],106.0,5.7,173.0,1995


Entretanto, esse formato não é útil para nós. Precisamos fazer algo chamado explode, que consiste em criar múltiplas cópias do filme, uma para cada gênero que ele possui, cada uma com um gênero. 

In [17]:
s = df.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre'
gen_df = df.drop('genres', axis=1).join(s)
gen_df.head()

/var/folders/8r/76rtgv650dq8578qqnhrpvym0000gn/T/ipykernel_12413/372891259.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = df.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)


,title,runtime,vote_average,vote_count,year,genre
0,Toy Story,81.0,7.7,5415.0,1995,animation
0,Toy Story,81.0,7.7,5415.0,1995,comedy
0,Toy Story,81.0,7.7,5415.0,1995,family
1,Jumanji,104.0,6.9,2413.0,1995,adventure
1,Jumanji,104.0,6.9,2413.0,1995,fantasy


Agora estamos prontos para criar uma função que irá agir como nosso recommender. Vamos criar os seguintes passos: 

* Obter o input do usuário sobre suas preferências
* Extrair todos os filmes que dão match com as condições do usuário
* Calcular o valor de $m$ e $C$ apenas para esses filmes e construir a lista de recomendação como vimos anteriormente

In [18]:
def build_chart(gen_df, percentile=0.8):
    
    print("Diga seu gênero preferido:")
    genre = input()

    print("Duração mínima:")
    low_time = int(input())
    
    print("Duração máxima:")
    high_time = int(input())

    print("Ano de lançamento mínimo")
    low_year = int(input())

    print("Ano de lançamento máximo")
    high_year = int(input())
    
    #Cria uma nova variável
    movies = gen_df.copy()
    
    #Filtra os filmes baseados nas condições fornecidas
    movies = movies[(movies['genre'] == genre) & 
                    (movies['runtime'] >= low_time) & 
                    (movies['runtime'] <= high_time) & 
                    (movies['year'] >= low_year) & 
                    (movies['year'] <= high_year)]
    
    #Calcula o valor de C e m
    C = movies['vote_average'].mean()
    m = movies['vote_count'].quantile(percentile)
    
    #considera apenas filmes com mais que m votos
    q_movies = movies.copy().loc[movies['vote_count'] >= m]
    
    #Calcula o score
    q_movies['score'] = q_movies.apply(lambda x: (x['vote_count']/(x['vote_count']+m) * x['vote_average']) 
                                       + (m/(m+x['vote_count']) * C)
                                       ,axis=1)

    #Ordena os filmes de maneira descendente
    q_movies = q_movies.sort_values('score', ascending=False)
    
    return q_movies

In [19]:
# testa a função
build_chart(gen_df).head()

Diga seu gênero preferido:


 action


Duração mínima:


 110


Duração máxima:


...

Ano de lançamento mínimo


 1990


Ano de lançamento máximo


 1999


,title,runtime,vote_average,vote_count,year,genre,score
2458,The Matrix,136.0,7.9,9079.0,1999,action,7.753691
582,Terminator 2: Judgment Day,137.0,7.7,4274.0,1991,action,7.448954
1460,The Fifth Element,126.0,7.3,3962.0,1997,action,7.102469
2800,Total Recall,113.0,7.1,1745.0,1990,action,6.797813
452,The Fugitive,130.0,7.2,1240.0,1993,action,6.784047


In [20]:
#salvando os dados para serem usados posteriormente
df.to_csv('dados/metadata_clean.csv', index=False)

## Filtros baseados em conteúdo

Os recommenders que construímos anteriormente são extremamente primitivos. O Simple Recommender não leva em consideração as preferências individuais e, enquanto o Knowledge Recommender leva em consideração a preferência do usuário por gêneros, época de lançamento e duração, o modelo continua ainda muito genérico. 

Por exemplo, se uma pessoa gosta dos filmes The Dark Knight, Homem de Ferro e Superman, claramente ele tem uma preferência por filmes de superheróis. Entretanto, nosso modelo não será capaz de capturar tais detalhes. O máximo que conseguirá fazer é fornecer recomendações de filmes de ação, o que é gênero maior que envolve os filmes de superheróis. 

Poderíamos solicitar ao usuário mais informações sobre seus gostos e criar, por exemplo, uma categoria sub-gênero. No entanto, não temos informação pra isso e capturar todas as informações necessárias via usuário seria maçante e muito custoso. 

Ao invés disso, o que os usuários costumam fazer é **classificar seus filmes favoritos e o sistema apresenta a eles os mais similares.** 

Aqui, vamos construir dois tipos de filtros baseados em conteúdo:

1. Recommender baseado na descrição dos filmes
    > Ele vai comparar a descrição de diferentes filmes e prover recomendações de filmes que possuam descrição similar

2. Recommender baseado em metadados
    > Ele usa como features gêneros, keywords, cast, entre outros e faz recomendações baseadas nessas características

#### Recommender baseado na descrição dos filmes

In [21]:
df = pd.read_csv('dados/metadata_clean.csv')
df.head()

,title,genres,runtime,vote_average,vote_count,year
0,Toy Story,"['animation', 'comedy', 'family']",81.0,7.7,5415.0,1995
1,Jumanji,"['adventure', 'fantasy', 'family']",104.0,6.9,2413.0,1995
2,Grumpier Old Men,"['romance', 'comedy']",101.0,6.5,92.0,1995
3,Waiting to Exhale,"['comedy', 'drama', 'romance']",127.0,6.1,34.0,1995
4,Father of the Bride Part II,['comedy'],106.0,5.7,173.0,1995


Essencialmente, os modelos que estamos construíndo calcula a similaridade entre os textos das descrições, mas como conseguimos quantificar a similaridade entre dois textos? 

Bem, vamos usar as técnicas que vimos rapidamente durante a aula: vetorização. Mais precisamente, vamos trabalhar com o TI-IDF. 

Basicamente, nosso modelo irá receber o título do filme como argumento e retornar uma lista de filmes similares baseada na descrição. Estes são os passos que vamos executar para construir nosso modelo:

* Obter os dados requeridos para construir o modelo
* Criar vetorização usando TF-IDF para cada filme
* Calcular a similaridade do cosseno par-a-par para cada filme
* Escrever uma função que recebe um título de filme como entrada e retorna uma lista de filmes similares

Olhando nossos dados, não temos a descrição dos filmes, mas isso é facilmente recuperado a partir dos dados originais. Vamos fazer isso:

In [22]:
orig_df = pd.read_csv('dados/movies_metadata.csv', low_memory=False)

#Adiciona as informações necessárias no nosso dataseet
df['overview'], df['id'] = orig_df['overview'], orig_df['id']
df.head()


,title,genres,runtime,vote_average,vote_count,year,overview,id
0,Toy Story,"['animation', 'comedy', 'family']",81.0,7.7,5415.0,1995,"Led by Woody, Andy's toys live happily in his ...",862
1,Jumanji,"['adventure', 'fantasy', 'family']",104.0,6.9,2413.0,1995,When siblings Judy and Peter discover an encha...,8844
2,Grumpier Old Men,"['romance', 'comedy']",101.0,6.5,92.0,1995,A family wedding reignites the ancient feud be...,15602
3,Waiting to Exhale,"['comedy', 'drama', 'romance']",127.0,6.1,34.0,1995,"Cheated on, mistreated and stepped on, the wom...",31357
4,Father of the Bride Part II,['comedy'],106.0,5.7,173.0,1995,Just when George Banks has recovered from his ...,11862


Vamos usar a coluna *overview* para construir esse modelo e a coluna *id* para construir o próximo.

O próximo passo agora é realizar a vetorização dos dados. Vamos usar Scikit-Learn para fazer isso:

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')

#Caso haja algum review vazio, é preenchido com uma string
df['overview'] = df['overview'].fillna('')

#constrói a matriz TF-IDF
tfidf_matrix = tfidf.fit_transform(df['overview'])
tfidf_matrix.shape #qtde de amostras, qtde de features

(45466, 75827)

O próximo passo é construir uma matriz que conterá a similaridade do cosseno par a par para cada filme, ou seja, vamos criar uma matriz de 45466x45466 em que a célula na linha $i$ e coluna $j$ representa a similaridade entre os filmes $i$ e $j$. Novamente, SKlearn vem para nos ajudar com uma função pronta:

In [24]:
from sklearn.metrics.pairwise import linear_kernel

# Calcula a matriz de similaridade do cosseno
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)


O passo final é construir a função de recomendação. Antes disso, porém, vamos um mapeamento entre índice e título de filmes para facilitar a recuperação do nome dos filmes

In [25]:
indices = pd.Series(df.index, index=df['title']).drop_duplicates()

In [26]:
def content_recommender(title, cosine_sim=cosine_sim, df=df, indices=indices):
    # obtém o índice do filme dado o título 
    idx = indices[title]

    # obtém o score de similaridade par a par de todos os filmes com o filme em questão
    # e converte numa lista de tuplas
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Ordena os filmes baseados na similaridade do cosseno
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Obtém o score dos dez mais similares, ignorando o primeiro(próprio filme).
    sim_scores = sim_scores[1:11]

    # Obtém o índice dos filmes
    movie_indices = [i[0] for i in sim_scores]

    # Retorna o título dos 10 mais similares
    return df['title'].iloc[movie_indices]

In [27]:
content_recommender('The Lion King')

34682    How the Lion Cub and the Turtle Sang a Song
9353                                The Lion King 1½
9115                  The Lion King 2: Simba's Pride
42829                                           Prey
25654                                 Fearless Fagan
17041                                   African Cats
27933              Massaï, les guerriers de la pluie
6094                                       Born Free
37409                                     Sour Grape
3203                                The Waiting Game
Name: title, dtype: object

Percebemos que nosso recommender funciona relativamente bem. A maioria das recomendações fazem sentido já que, de uma maneira ou outra, trata de filmes com leões em suas descrições. Entretanto, quem assistiu o Rei Leão, provavelmente gostaria de ver recomendação de filmes das disney, mas nosso recommender não consegue capturar essas nuances. 

Felizmente, temos uma maneira de resolver esse problema. É o que vamos fazer com o Recommender baseado em metadados

#### Recommender baseado em metadados
Basicamente, vamos seguir os mesmos passos na criação de nosso recommender. Entretanto, os dados a serem usados serão diferentes. Basicamente, vamos usar os seguintes metadados:

* Gênero  do filme
* O diretor do filme
* Três maiores estrelas do filme
* Sub-gêneros ou keywords

Para isso, vamos precisar de dois arquivos adicionais:

In [28]:
cred_df = pd.read_csv('dados/credits.csv')
key_df = pd.read_csv('dados/keywords.csv')


In [29]:
cred_df.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [30]:
len(key_df)

46419

In [31]:
key_df.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


Podemos ver que as informações do cast, equipe técnica (crew) e keywords estão na forma de lista de dicionários. Assim como fizemos anteriormente com gênero, precisamos reduzie isso a uma string ou lista de strings. 

Antes disso, porém, vamos agrupar nossos três DataFrames de modo que todas as nossas features estejam num lugar só. Para fazer o join, vamos o *id* e, para evitar erros, vamos tratar os ids, já que temos "1997-08-20" como ID.  

In [32]:
def clean_ids(x):
    try:
        return int(x)
    except:
        return np.nan

In [33]:
df['id'] = df['id'].apply(clean_ids)
df = df[df['id'].notnull()]

Agora podemos converter todos os IDs em int e agrupar os DataFrames

In [34]:
df['id'] = df['id'].astype(int)
key_df['id'] = key_df['id'].astype(int)
cred_df['id'] = cred_df['id'].astype(int)

df = df.merge(cred_df, on='id')
df = df.merge(key_df, on='id')
df.head()


/var/folders/8r/76rtgv650dq8578qqnhrpvym0000gn/T/ipykernel_12413/571838465.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['id'] = df['id'].astype(int)


,title,genres,runtime,vote_average,vote_count,year,overview,id,cast,crew,keywords
0,Toy Story,"['animation', 'comedy', 'family']",81.0,7.7,5415.0,1995,"Led by Woody, Andy's toys live happily in his ...",862,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,Jumanji,"['adventure', 'fantasy', 'family']",104.0,6.9,2413.0,1995,When siblings Judy and Peter discover an encha...,8844,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,Grumpier Old Men,"['romance', 'comedy']",101.0,6.5,92.0,1995,A family wedding reignites the ancient feud be...,15602,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,Waiting to Exhale,"['comedy', 'drama', 'romance']",127.0,6.1,34.0,1995,"Cheated on, mistreated and stepped on, the wom...",31357,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...","[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,Father of the Bride Part II,['comedy'],106.0,5.7,173.0,1995,Just when George Banks has recovered from his ...,11862,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


Agora que temos todas as features num único Dataframe, vamos transformá-las num formáto útil. Especificamente, essas são as transformações que iremos executar:

* Converter *keywords* numa lista de string (vamos inclur somente as top 3)
* Converter *cast* numa lista de string (vamos incluir somente as top 3)
* De *crew*, vamos recuperar somente o diretor

In [35]:
#convertendo em objetos python
features = ['cast', 'crew', 'keywords', 'genres']
for feature in features:
    df[feature] = df[feature].apply(literal_eval)

Para extrair o nome do diretor, vamos analisar um objeto:

In [36]:
df.iloc[0]['crew'][0]

{'credit_id': '52fe4284c3a36847f8024f49',
 'department': 'Directing',
 'gender': 2,
 'id': 7879,
 'job': 'Director',
 'name': 'John Lasseter',
 'profile_path': '/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg'}

Assim, vamos criar uma função que olhe somente para a key 'name' e recupera a informação desejada. 

In [37]:
def get_director(x):
    for crew_member in x:
        if crew_member['job'] == 'Director':
            return crew_member['name']
    return np.nan

In [38]:
df['director'] = df['crew'].apply(get_director)
df['director'].head()

0      John Lasseter
1       Joe Johnston
2      Howard Deutch
3    Forest Whitaker
4      Charles Shyer
Name: director, dtype: object

*keywords* e *cast* são lista de dicionários e precisamos extrair o top 3 do atributo 'names'. Vamos escrever uma função que faça isso:

In [39]:
def generate_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        #Se tiver mais que tres, retorna somente os tres primeiros
        if len(names) > 3:
            names = names[:3]
        return names

    #retorna uma lista vazia, caso nao tenha tres
    return []

In [40]:
df['cast'] = df['cast'].apply(generate_list)
df['keywords'] = df['keywords'].apply(generate_list)

In [41]:
#recuperando somentes os top 3 generos
df['genres'] = df['genres'].apply(lambda x: x[:3])

In [42]:
#vamos verificar como ficou nosso DataFrame
df[['title', 'cast', 'director', 'keywords', 'genres']].head()

,title,cast,director,keywords,genres
0,Toy Story,"[Tom Hanks, Tim Allen, Don Rickles]",John Lasseter,"[jealousy, toy, boy]","[animation, comedy, family]"
1,Jumanji,"[Robin Williams, Jonathan Hyde, Kirsten Dunst]",Joe Johnston,"[board game, disappearance, based on children'...","[adventure, fantasy, family]"
2,Grumpier Old Men,"[Walter Matthau, Jack Lemmon, Ann-Margret]",Howard Deutch,"[fishing, best friend, duringcreditsstinger]","[romance, comedy]"
3,Waiting to Exhale,"[Whitney Houston, Angela Bassett, Loretta Devine]",Forest Whitaker,"[based on novel, interracial relationship, sin...","[comedy, drama, romance]"
4,Father of the Bride Part II,"[Steve Martin, Diane Keaton, Martin Short]",Charles Shyer,"[baby, midlife crisis, confidence]",[comedy]


Por último, precisamos criar uma função simples para preparar esses textos. Vamos remover os espaços em branco dos nomes de cast e director (de modo que Ryan Reynolds e Ryan Gosling sejam tratados de forma diferente) e colocar todos os textos em minúsculas. 

In [43]:
def sanitize(x):
    if isinstance(x, list):
        #remove espaço em branco e coloca em minúscula
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #s nao tiver diretor, retorna uma string vazia
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [44]:
for feature in ['cast', 'director', 'genres', 'keywords']:
    df[feature] = df[feature].apply(sanitize)

Antes de aplicar a vetorização, no entanto, precisamos concatenar todas as features que temos numa só. 

In [45]:
def create_feat(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])

In [46]:
df['feature'] = df.apply(create_feat, axis=1)

In [47]:
df.iloc[0]['feature']

'jealousy toy boy tomhanks timallen donrickles johnlasseter animation comedy family'

Agora estamos prontos para realizar a vetorização. Ao invés de usar TF-IDF, vamos usar o BoW (obtido através de CountVectorizer). Isso porque o TFIDI irá dar um peso menor para atores e diretores que atuaram e dirigiram um grande número de filmes.

In [48]:
df['feature'][0:40000]

0        jealousy toy boy tomhanks timallen donrickles ...
1        boardgame disappearance basedonchildren'sbook ...
2        fishing bestfriend duringcreditsstinger walter...
3        basedonnovel interracialrelationship singlemot...
4        baby midlifecrisis confidence stevemartin dian...
                               ...                        
39995     kelliemartin martincummins cindysampson stefa...
39996    londonengland tattoo assassin alecbaldwin powe...
39997    suspense jimbelushi angelafeatherstone jasonba...
39998            christinelahti campbellscott alisonpill  
39999    rebel usapresident hostage marielhemingway dav...
Name: feature, Length: 40000, dtype: object

In [49]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(df['feature']) #se der problema de memória, reduza a quantidade de linhas: df['feature'][0:30000]


In [50]:
#calcula a similaridade par a par
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

Construindo o mapeamento para recuperar o nome dos filmes

In [51]:
df = df.reset_index()
indices2 = pd.Series(df.index, index=df['title'])

Agora estamos prontos para usar nosso recommender

In [52]:
content_recommender('The Lion King', cosine_sim2, df, indices2)

29607                                          Cheburashka
40904                   VeggieTales: Josh and the Big Wall
40913    VeggieTales: Minnesota Cuke and the Search for...
27768                                 The Little Matchgirl
15209             Spiderman: The Ultimate Villain Showdown
16613                            Cirque du Soleil: Varekai
24654                                  The Seventh Brother
29198                                      Superstar Goofy
30244                                              My Love
31179                Pokémon: Arceus and the Jewel of Life
Name: title, dtype: object

As recomendações nesse caso foram muito diferentes das anteriores. Vimos que nosso recommender foi capaz de capturar mais informações que apenas 'leões'. Muitos dos filmes na lista são animações e dizem respeito a personagens antropomórficos. 

## Filtros Colaborativos

Vamos agora entender como funcionam os filtros colaborativos. Primeiramente,vamos construir um framework bem definido que permitirá a construção e teste de nossos filtros colaborativos sem esforço. O framework consiste dos dados, métrica de avaliação e uma função que calcula a métrica para um dado modelo. 

Para este caso, precisamos que os dados contenham comportamente do usuário. Por isso, vamos usar outro dataset: MovieLens [link](https://www.kaggle.com/datasets/prajitdatta/movielens-100k-dataset)

Vamos usar as seguintes partes: u.data, u.user e u.item. 

In [53]:
use_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']

users = pd.read_csv('dados/u.user', sep='|', names=use_cols,encoding='latin-1')
users.head() #contém dados demográficos dos usuários

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [54]:
i_cols = ['movie_id', 'title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

movies = pd.read_csv('dados/u.item', sep='|', names=i_cols, encoding='latin-1')
movies.head() #contém informacoes acerca dos filmes que foram classificados pelos usuários

,movie_id,title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


Como estamos trabalhando com filtros colaborativos, a única coisa de interesse são o nome do filme e seu id

In [55]:
movies = movies[['movie_id', 'title']]

In [56]:
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']

ratings = pd.read_csv('dados/u.data', sep='\t', names=r_cols, encoding='latin-1')
ratings.head() #contém as classificacoes que cada usuário deu a cada filme. É a partir desse arquivo que vamos construir a
#matrix de ratings

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


Nesse caso, não estamos interessados no exato momento em que o usuário deu o rating para o filme, então eliminamos essa coluna

In [57]:
ratings = ratings.drop('timestamp', axis=1)

Agora podemos modelar nosso problema como um problema de regressão, em que nosso modelo seja capaz de predizer o id de um usuário, ou seja, a partir do filme assistido e de seu respectivo rating, vamos tentar predizer o usuário. Vamos fazer isso:

In [58]:
from sklearn.model_selection import train_test_split

X = ratings.copy()
y = ratings['user_id']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, stratify=y, random_state=42)

Para avaliar nosso modelo, vamos usar a já conhecida RMSE. obtida através da seguinte funcão:

In [59]:
from sklearn.metrics import mean_squared_error
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

Vamos definir agora um baseline para nosso modelo. O que o nosso filtro colaborativo vai receber como input é o *user_id* e o *movie_id* e retornar um float entre 1 e 5. Nosso baseline define, independentemente do *user_id* e *movie_id*, o retorno será 3.

In [60]:
def baseline(user_id, movie_id):
    return 3.0

Vamos verificar como nosso modelo se sai usando o baseline como resposta correta para todos os usuários:

In [61]:
def score(cf_model):
    
    #constroi uma lista de usuario-filme a partir do conjunto de teste
    id_pairs = zip(X_test['user_id'], X_test['movie_id'])
    
    #prediz o rating para tupla usuario-filme
    y_pred = np.array([cf_model(user, movie) for (user, movie) in id_pairs])
    
    #recupera o rating correto
    y_true = np.array(X_test['rating'])
    
    #retorna o RMSE final
    return rmse(y_true, y_pred)

In [62]:
score(baseline)

1.2488234462885457

Nosso objetivo se resume a melhorar esse score

Para comecarmos a construir os filtros colaborativos, vamos criar uma matriz de ratings usando pivot_table, do Pandas.

In [63]:
r_matrix = X_train.pivot_table(values='rating', index='user_id', columns='movie_id')
r_matrix.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,1671,1672,1673,1674,1676,1677,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,NaN,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Temos agora uma matriz em cada linha é um usuário e cada coluna um filme. Muitos valores NaN representam filmes que os usuários nao classificaram. Isso denota alta esparsidade dos dados. 

Vamos construir um simples filtro colaborativo, que retorna um rating médio de um filme baseado na classificacao que todos os usuários deram. 

OBS: é possível que alguns filmes tenham sido avaliados apenas no conjunto de teste e nao no conjunto de treino. Para esses casos, vamos usar o baseline.

In [64]:
def cf_user_mean(user_id, movie_id):
    
    #Check if movie_id exists in r_matrix
    if movie_id in r_matrix:
        #Compute the mean of all the ratings given to the movie
        mean_rating = r_matrix[movie_id].mean()
    
    else:
        #Default to a rating of 3.0 in the absence of any information
        mean_rating = 3.0
    
    return mean_rating

In [65]:
score(cf_user_mean)

1.0300824802393536

Ok, conseguimos melhorar o RMSE. Entretanto, neste modelo, atribuímos pesos iguais a todos os usuários. Entretanto, é mais intuitivo dar mais preferência aos ratings de usuários que são similares ao usuário em questão. 

Vamos usar a similiradidade do cosseno para obter a similaridade entre cada usuário. Como a funcao da sikit-learn nao aceita NaN como input, vamos substituí-los por 0. 

In [66]:
r_matrix_dummy = r_matrix.copy().fillna(0)

In [67]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(r_matrix_dummy, r_matrix_dummy)

In [68]:
#converte num Pandas DataFrame
cosine_sim = pd.DataFrame(cosine_sim, index=r_matrix.index, columns=r_matrix.index)
cosine_sim.head(10)

user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.108361,0.046638,0.029577,0.245753,0.335853,0.344724,0.191582,0.057149,0.251979,...,0.257073,0.069412,0.231643,0.108093,0.176842,0.104799,0.232472,0.051528,0.129555,0.256333
2,0.108361,1.000000,0.057613,0.130237,0.054918,0.190552,0.079399,0.076146,0.167992,0.147376,...,0.136993,0.252887,0.255454,0.285193,0.232751,0.149088,0.102807,0.062386,0.109143,0.107686
3,0.046638,0.057613,1.000000,0.139805,0.000000,0.032485,0.043869,0.080968,0.022263,0.059925,...,0.027402,0.000000,0.175060,0.010343,0.105635,0.019052,0.127099,0.023917,0.060392,0.000000
4,0.029577,0.130237,0.139805,1.000000,0.000000,0.045190,0.088586,0.199526,0.135013,0.026919,...,0.055392,0.049773,0.076549,0.139382,0.113886,0.000000,0.130343,0.077357,0.157890,0.063911
5,0.245753,0.054918,0.000000,0.000000,1.000000,0.176443,0.281860,0.132205,0.038790,0.134200,...,0.183969,0.019305,0.073714,0.041807,0.081088,0.029743,0.188392,0.068342,0.055557,0.207259
6,0.335853,0.190552,0.032485,0.045190,0.176443,1.000000,0.394725,0.143385,0.125126,0.372679,...,0.328643,0.070809,0.135806,0.171670,0.125446,0.086464,0.230566,0.095478,0.197307,0.185268
7,0.344724,0.079399,0.043869,0.088586,0.281860,0.394725,1.000000,0.215861,0.121224,0.378723,...,0.339853,0.110866,0.096055,0.104690,0.126108,0.075012,0.270071,0.020036,0.236086,0.266571
8,0.191582,0.076146,0.080968,0.199526,0.132205,0.143385,0.215861,1.000000,0.116173,0.169088,...,0.150048,0.064242,0.118297,0.053969,0.168057,0.095736,0.164157,0.076269,0.089871,0.210995
9,0.057149,0.167992,0.022263,0.135013,0.038790,0.125126,0.121224,0.116173,1.000000,0.152694,...,0.082819,0.064400,0.127051,0.069251,0.095673,0.000000,0.131458,0.106763,0.089297,0.089583


Agora estamos prontos para calcular o score usando essa matrix como entrada e ver se reduzimos o RMSE. 

In [69]:
def cf_user_wmean(user_id, movie_id):
    
    #verifica se o filme tá na matriz
    if movie_id in r_matrix:
        
        #obtém o score de similaridade do usuário com todos os outros usuários
        sim_scores = cosine_sim[user_id]
        
        #obtém o rating do usuário para o filme em questao
        m_ratings = r_matrix_dummy[movie_id]
        
        #obtém os indices que tenham NaN
        idx = m_ratings[m_ratings.isnull()].index
        
        #Dropa NaN
        m_ratings = m_ratings.dropna()
        
        #Dropa os scores 
        sim_scores = sim_scores.drop(idx)
        
        #calcular a média ponderada
        wmean_rating = np.dot(sim_scores, m_ratings)/ sim_scores.sum()
    
    else:
        #se o filme nao ta na matriz, retorna o baseline
        wmean_rating = 3.0
    
    return wmean_rating

In [70]:
score(cf_user_wmean)

3.041758008854614

Não foi o resultado desejado, então vamos testar o uso de dados demográficos para ver se conseguimos um resultado melhor.

Diferentemente dos anteriores, esse filtro não leva em consideração os ratings dos filmes. Vamos começar construindo um filtro baseado em gênero. 

In [71]:
#vamos obter as informacoes demográficas do dataset users e concatenar com o X_train
merged_df = pd.merge(X_train, users)
merged_df.head()

,user_id,movie_id,rating,age,sex,occupation,zip_code
0,862,177,4,25,M,executive,13820
1,862,416,3,25,M,executive,13820
2,862,1093,5,25,M,executive,13820
3,862,168,4,25,M,executive,13820
4,862,568,3,25,M,executive,13820


Agora precisamos calcular o rating médio de cada filme por gênero. Vamos usar groupby

In [72]:
gender_mean = merged_df[['movie_id', 'sex', 'rating']].groupby(['movie_id', 'sex'])['rating'].mean()
gender_mean

movie_id  sex
1         F      3.797872
          M      3.888446
2         F      3.285714
          M      3.202703
3         F      2.916667
                   ...   
1677      F      3.000000
1679      M      3.000000
1680      M      2.000000
1681      M      3.000000
1682      M      3.000000
Name: rating, Length: 3047, dtype: float64

Agora estamos prontos para construir uma função que identifica o gênero do usuário, extrai o rating médio dado o filme e retorna esse valor como output:

In [73]:
#configura o index do dataframe para user_id
users = users.set_index('user_id')

In [74]:
def cf_gender(user_id, movie_id):
    
    #verifica se o filme existe
    if movie_id in r_matrix:
        #Identifica o gênero do usuário
        gender = users.loc[user_id]['sex']
        
        #verifica se o gênero classificou o filme
        if gender in gender_mean[movie_id]:
            
            #calcula o rating médio do filme pelo genero
            gender_rating = gender_mean[movie_id][gender]
        
        else:
            gender_rating = 3.0
    
    else:
        #Default
        gender_rating = 3.0
    
    return gender_rating

In [75]:
score(cf_gender)

1.0392906999935203